In [2]:
# https://huggingface.co/transformers/v3.2.0/custom_datasets.html

In [3]:
!pip install transformers -U

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [4]:
import pandas as pd

In [5]:
# https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/data

data = pd.read_excel("/kaggle/input/bart-project-data/PROJECT_DATA.xlsx")

data.head()

,SNO.,MCV,MCHC,HB,RBC,WBC,PLT,RDWCV,NEUTRO,LYMPHO,SUMMARY
0,Report1,87.7,30.1,7.3,2.77,10.0,189.0,11.4,50.1,43.2,Your report shows a hemoglobin level of 7.3 g/...
1,Report2,88.2,20.2,7.3,2.84,10.0,180.0,11.4,52.3,42.4,"In your CBC report, your hemoglobin level is 7..."
2,Report3,77.0,29.5,9.0,3.97,7.2,148.0,13.7,60.7,30.7,"In your CBC report, your hemoglobin level is 9..."
3,Report4,77.9,29.8,3.8,4.22,6.0,143.0,17.0,63.5,30.2,"Your CBC shows a hemoglobin level of 3.8 g/dL,..."
4,Report5,80.6,29.7,0.4,3.93,4.2,236.0,15.1,53.7,39.1,"In your CBC, the hemoglobin level is criticall..."


In [6]:
# data.to_csv("/content/drive/MyDrive/Youtube Tutorials/datasets/toxic_commnets.csv",index=False)

In [7]:
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

import torch

from transformers import TrainingArguments, Trainer

from transformers import BertTokenizer, BertForSequenceClassification

In [8]:
pip install datasets


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [9]:
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments, TrainerCallback
from datasets import Dataset
import torch

# Load the pre-trained BART model for summarization
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Preprocessing function
def preprocess_function(examples):
    input_texts = [
        " ".join([
            str(examples['MCV '][i]),
            str(examples['MCHC '][i]),
            str(examples['HB '][i]),
            str(examples['RBC '][i]),
            str(examples['WBC '][i]),
            str(examples['PLT'][i]),
            str(examples['RDWCV'][i]),
            str(examples['NEUTRO'][i]),
            str(examples['LYMPHO'][i])
        ])
        for i in range(len(examples['MCV ']))
    ]
    model_inputs = tokenizer(input_texts, max_length=1024, padding="max_length", truncation=True)
    labels = tokenizer(examples['SUMMARY '], max_length=200, padding="max_length", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Create Dataset object from DataFrame (assuming 'data' is your DataFrame)
dataset = Dataset.from_pandas(data)
tokenized_datasets = dataset.map(preprocess_function, batched=True)
train_test_split_datasets = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_test_split_datasets['train']
test_dataset = train_test_split_datasets['test']

# Define TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=70,  # Train for 70 epochs
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="loss",
)

# Custom callback to log only the first 3 epochs
class LimitedLoggingCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.epoch > 3:
            control.should_log = False  # Disable logging after 3 epochs

# Set up the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[LimitedLoggingCallback()]  # Add the custom callback here
)

# Train the model
trainer.train()

# Evaluate the model on the test set
results = trainer.evaluate()
print("Test Results:", results)


Map:   0%|          | 0/282 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jheel-turakhia-btech2022 (jheel-turakhia-btech2022-symbiosis-international). Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,6.201600,5.163369
2,3.511400,1.948184
3,1.466200,1.008003


/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked t

Test Results: {'eval_loss': 1.0080029964447021, 'eval_runtime': 5.7467, 'eval_samples_per_second': 9.919, 'eval_steps_per_second': 1.392, 'epoch': 3.0}
The average blood pressure is at 85.85 fLFV-CV, which is within the normal range. The red blood cell count is also within normal limits. The hemoglobin level is at 9.85 g/dL, which indicates good oxygen transport capacity. The platelet count is at 8.85 million cells/µL, which may indicate a healthy immune response. The white blood cell size is at 7.85%, which is healthy. The neutrophils are at a healthy level, with a low neutrophil count of 1.85%. The platelets are also at normal levels, which shows good immune function.


In [14]:
# Step 10: Save the model and tokenizer (optional)
model.save_pretrained("./model")
tokenizer.save_pretrained("./model")


('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.json',
 './model/merges.txt',
 './model/added_tokens.json')

In [11]:
!pip install evaluate


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [12]:
pip install rouge-score

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import evaluate
import numpy as np

# Load the ROUGE metric
rouge_metric = evaluate.load("rouge")

# Updated compute_metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Convert predictions and labels to lists of token IDs
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    # Flatten predictions and labels (avoid nested lists)
    predictions = [item for sublist in predictions for item in sublist if item is not None]
    labels = [item for sublist in labels for item in sublist if item is not None]

    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Check if any decoded value is None and handle it
    decoded_preds = [pred if pred is not None else "" for pred in decoded_preds]
    decoded_labels = [label if label is not None else "" for label in decoded_labels]

    # Rouge expects newline after each sentence
    decoded_preds = ["\n".join(pred.strip().split()) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.strip().split()) for label in decoded_labels]

    # Compute ROUGE scores
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Average the scores across examples
    result = {key: np.mean([score.fmeasure for score in value]) for key, value in result.items()}

    return result

# Now redefine the trainer with the updated compute_metrics function
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Evaluate the model
results = trainer.evaluate()

# Print the evaluation results with ROUGE scores
print("Evaluation Results:", results)
